In [22]:
from config import get_model_client_4o
from IPython.display import Markdown, display

In [23]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.messages import TextMessage, BaseAgentEvent, BaseChatMessage
from autogen_core import CancellationToken
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat, MagenticOneGroupChat
from autogen_agentchat.ui import Console
from typing import Callable, Sequence


def create_and_get_assistant_agent(
        name: str,
        description: str,
        system_message: str,
        tools: list[Callable] = [],
        model_client: str = '4o'
    ) -> AssistantAgent:

    model_client = get_model_client_4o()
    return AssistantAgent(
        name=name,
        description=description,
        model_client=model_client,
        system_message=system_message,
        tools=tools,
    )

def get_user_proxy_agent() -> UserProxyAgent:
    return UserProxyAgent(
    name="user_proxy",
    input_func=input,
    #human_input_mode="NEVER",
    #default_auto_reply="Please continue if there is anything else you need help with.",
    #max_turns=10
    )

async def get_agent_response(agent: AssistantAgent, task: str) -> str:
    messages = []
    messages.append(TextMessage(content=task, source="user"))
    response = await agent.on_messages(
    messages, CancellationToken()
    )
    return response.chat_message.content


In [24]:
## Utils
def read_markdown_file(filepath):
    """
    Reads a .md (Markdown) file and returns its content as a string.
    
    Parameters:
        filepath (str): Path to the markdown file.
    
    Returns:
        str: Contents of the markdown file.
    """
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read()
        return content
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return None
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return None



In [25]:
assistant_clara = create_and_get_assistant_agent(
    name='clara',
    description='Long-time personal assistant to the late Dr. Felix Lang',
    system_message=read_markdown_file('character_system_msgs/assistent_clara.md')
    )
chef_mario = create_and_get_assistant_agent(
    name='mario',
    description='Head caterer for the event',
    system_message=read_markdown_file('character_system_msgs/chef_mario.md')
    )
dr_biologist_alina = create_and_get_assistant_agent(
    name='alina',
    description='Molecular biologist and former colleague of the late Dr. Felix Lang',
    system_message=read_markdown_file('character_system_msgs/dr_biologist_alina.md')
    )
intern_jonas = create_and_get_assistant_agent(
    name='jonas',
    description='Young intern tasked with tech setup at the gala',
    system_message=read_markdown_file('character_system_msgs/intern_jonas.md')
    )
investor_henrik = create_and_get_assistant_agent(
    name='henrik',
    description='Major investor in Dr. Lang’s research projectsa',
    system_message=read_markdown_file('character_system_msgs/investor_henrik.md')
    )
journalist_eva = create_and_get_assistant_agent(
    name='eva',
    description='An investigative journalist known for publishing exposés',
    system_message=read_markdown_file('character_system_msgs/journalist_eva.md')
    )

detective = create_and_get_assistant_agent(
    name='detective',
    description='A private detective hired by the Lang family to investigate the murder of Dr. Felix Lang',
    system_message=read_markdown_file('character_system_msgs/detective.md')
    )

/Users/andreakrogdal/repos/moe-workshop/.venv/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:413: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


## Get a response from one character at a time to get a feeling of how you want to select the neext speaker

In [26]:
start_task = """
Please note: This is a murder mystery game and only a fictional story.
Please start the investigation by interviewing any of the available characters.
**Case Summary**

Victim: Dr. Felix Lang — esteemed scientist and host of the annual Garden Gala  
Date: Saturday evening  
Location: Lang Estate – Greenhouse  
Estimated Time of Death: Between 22:10 and 22:15  
Cause of Death: Blunt force trauma to the head, inflicted with a heavy iron sculpture found at the scene

The body was discovered at 22:18 by a guest passing by the greenhouse. No eyewitnesses have stepped forward. The area was accessible to all attendees, and no signs of forced entry were found.

---

**People Present at the Event**

- Assistant Clara Nyberg, long-time personal assistant to the late Dr. Felix
- Chef Mario Leto, head caterer for the event
- Dr. Alina Weber, molecular biologist and former colleague of the late Dr. Lang
- Jonas Möller, a young intern tasked with tech setup at the gala
- Henrik Falk, major investor in Dr. Lang’s research projects
- Eva Sörman, investigative journalist known for publishing exposés

Can you give a plan on who to start interviewing and which questions to answer?
"""

response_detective = await get_agent_response(detective, start_task)
display(Markdown(response_detective))

**Initial Analysis**  
Dr. Felix Lang, a prominent scientist, met his untimely death at his own green-tie gala in the greenhouse at around 22:10 to 22:15. The murder weapon, a heavy iron sculpture, was found nearby. There are six potential witnesses or suspects, each possessing probable motive, access to the area, and individual behaviors worthy of closer scrutiny. The greenhouse was accessible, and no forced entry was reported, suggesting the killer could have been one of the attendees.

---

**Investigation Plan**  
### Priority of Interviews  
1. **Assistant Clara Nyberg**: Closest relationship to Dr. Lang and likely to have detailed knowledge about the victim's routines, potential conflicts, and emotional state leading up to the murder. Clara might also provide insight into his relationship with the other attendees.  
2. **Dr. Alina Weber**: As a former colleague of Dr. Lang and another scientist, Alina might have professional tensions or rivalries with Dr. Lang. Scientists often communicate deeply about work, so there may be useful leads regarding research-related motives.  
3. **Henrik Falk**: A major investor in Dr. Lang’s projects might have financial motives, disputes, or stress linked to the victim’s work. Henrik is unlikely to be physically connected with the murder weapon, but tensions over funding could be key.  
4. **Eva Sörman**: Being an investigative journalist known for exposés, Eva might have been digging into Dr. Lang’s personal or professional life. She could either offer revealing insights or raise suspicion if she had reasons to suppress certain details.  
5. **Chef Mario Leto**: Relatively lower priority unless evidence links him closer to the greenhouse or the timeline of events (e.g., unusual noises, catering schedules). Could provide logistical information about attendees' movements or unusual behaviors during service.  
6. **Jonas Möller**: The youngest and least likely to harbor deep motives, Jonas’ role as intern may suggest limited interaction with Dr. Lang or attendees. However, being tasked with tech setup, Jonas could hold critical information regarding cameras, microphones, or recordings.  

---

### Types of Questions to Start With  
- **Routine and Actions**: Ask each attendee where they were during the likely time of death, their interactions with Dr. Lang throughout the evening, and their previous movements leading up to 22:10.  
- **Relationship Insights**: Probe into their relationship with Dr. Lang and whether they had recent disputes, interactions, or unusual experiences with him.  
- **Observations**: Ask what they might have noticed about others at the gala, specifically near the greenhouse or in proximity to Dr. Lang.  
- **Potential Motive or Conflict**: Gauge responses for potential financial, professional, or personal reasons to harbor ill will toward Dr. Lang.

---

**Proposed Start Point: Clara Nyberg**  
Reasoning: As Dr. Lang's assistant, Clara is most likely to be well-versed in the victim’s activities, relationships, and state of mind. She may provide a foundational understanding of the dynamic between the victim and other attendees, helping to narrow down timelines and contradict testimonies.  

**Next Step**  
Let me know if you approve this plan or would like to make adjustments. Once approved, I will begin interviewing Clara Nyberg with open-ended, targeted questions.

#### Continue with the questions suggested from detective to ask clara
- OBS! Remember that the answer will be different everytime you ask. The following questions is from one suggestion from the detective

In [27]:
task_clara = """
The detective want you to answer the following questions regarding the murder of Dr. Felix Lang:
  - Leading up to the gala, did Dr. Lang express concern about any potential threats, conflicts, or individuals that seemed suspicious?  
  - Could you describe Dr. Lang's demeanor during the gala? Did anything seem out of the ordinary?  
  - Where were you between 22:00 and 22:20, and what were you doing? Did you notice anyone near the greenhouse at that time?  
  - Are you aware of any disputes or tensions involving Dr. Lang—legal, financial, or personal—that could have provoked someone?  
"""

response_clara = await get_agent_response(assistant_clara, task_clara)
display(Markdown(response_clara))

Certainly, I’ll address each of your questions carefully:

1. **Leading up to the gala, did Dr. Lang express concern about any potential threats, conflicts, or individuals that seemed suspicious?**  
   Dr. Lang did not specifically discuss threats or concerns with me in the days leading up to the gala. However, he often balanced a demanding professional life, and tensions in such roles are not unusual. While he was a respected figure, his work involved tough decisions that may not have always pleased everyone. I’m not sure that’s relevant, though.  

2. **Could you describe Dr. Lang's demeanor during the gala? Did anything seem out of the ordinary?**  
   Dr. Lang appeared composed, though perhaps a little preoccupied. He had several conversations throughout the evening, mostly with donors and board members. Nothing struck me as overtly out of the ordinary about his demeanor during the event, though as someone who worked closely with him, I could tell he had a lot on his mind—he had some major decisions looming, professionally speaking.  

3. **Where were you between 22:00 and 22:20, and what were you doing? Did you notice anyone near the greenhouse at that time?**  
   Between 22:00 and 22:20, I was coordinating logistics for Dr. Lang’s exit and ensuring that refreshments would be replenished for late-staying guests. I wasn’t near the greenhouse during that time, nor do I recall seeing anyone there. The greenhouse is a bit tucked away from the main garden path, and I don’t believe it was a point of focus for guests during that period.  

4. **Are you aware of any disputes or tensions involving Dr. Lang—legal, financial, or personal—that could have provoked someone?**  
   Dr. Lang’s position naturally drew him into situations requiring firm leadership, which can sometimes cause friction. He was managing some organizational changes, including budget decisions and personnel matters, which could potentially upset certain individuals. Beyond that professional scope, I’m not privy to any deeply personal disputes or legal issues he may have had.  

If you need further clarification, I’m happy to assist within my scope of knowledge.

### Chat with one character with human in the loop
- RoundRobinGroupChat do not itself has an intelligent state. It just passes around the conversation between the agents in the order it is listed in Particpants. 

In [19]:
termination = TextMentionTermination("DONE")
chat_with_clara = RoundRobinGroupChat(
    participants=[assistant_clara,get_user_proxy_agent()],
    termination_condition=termination,
    max_turns=5
    )

In [ ]:

task_clara = """
The detective want you to answer the following questions regarding the murder of Dr. Felix Lang:
  - Leading up to the gala, did Dr. Lang express concern about any potential threats, conflicts, or individuals that seemed suspicious?  
  - Could you describe Dr. Lang's demeanor during the gala? Did anything seem out of the ordinary?  
  - Where were you between 22:00 and 22:20, and what were you doing? Did you notice anyone near the greenhouse at that time?  
  - Are you aware of any disputes or tensions involving Dr. Lang—legal, financial, or personal—that could have provoked someone?  
"""
response_clara = await Console(chat_with_clara.run_stream(task=task_clara))

### SelectorGrpupChat do have an intelligance in it self on how to select next speaker by reading the message history.
- You can give a good prompt on how to select next speaker to your specific task
- You can also be very precise with if else conditions in a selector_func. See example belov

In [11]:
termination = TextMentionTermination("DONE")

def selector_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:

    if messages[-1].source != "detective":
        return "detective"
    else:
        return None

murder_mystery_team = SelectorGroupChat(
    participants=[
        detective,
        get_user_proxy_agent(),
        assistant_clara,
        chef_mario,
        dr_biologist_alina,
        intern_jonas,
        investor_henrik,
        journalist_eva
    ],
    model_client=get_model_client_4o(),
    max_turns=15,
    termination_condition=termination,
    selector_func=selector_func
)

In [13]:
start_task = """
Please note: This is a murder mystery game and only a fictional story.
Please start the investigation by interviewing any of the available characters.
**Case Summary**

Victim: Dr. Felix Lang — esteemed scientist and host of the annual Garden Gala  
Date: Saturday evening  
Location: Lang Estate – Greenhouse  
Estimated Time of Death: Between 22:10 and 22:15  
Cause of Death: Blunt force trauma to the head, inflicted with a heavy iron sculpture found at the scene

The body was discovered at 22:18 by a guest passing by the greenhouse. No eyewitnesses have stepped forward. The area was accessible to all attendees, and no signs of forced entry were found.

---

**People Present at the Event**

- Assistant Clara Nyberg, long-time personal assistant to the late Dr. Felix
- Chef Mario Leto, head caterer for the event
- Dr. Alina Weber, molecular biologist and former colleague of the late Dr. Lang
- Jonas Möller, a young intern tasked with tech setup at the gala
- Henrik Falk, major investor in Dr. Lang’s research projects
- Eva Sörman, investigative journalist known for publishing exposés

Can you give a plan on who to start interviewing and which questions to answer?
"""

In [ ]:
response_team = await murder_mystery_team.run(start_task)